## NN Structure

output policy will have a dimension of the sum of discard choices, play choices, and clue choices. clue choices has a cardinality of product of num players, num colors, and num numbers. In this case it will be 4+4+3*3*1= 17 element array

Input layer will have a dimension of the sum of opp_hand, board, discard, self_clue, and opp_clue.
opp_hand will be dummy encoding of the number of cards in the deck. board is the number of playable cards. self_clue is product of num of cards_in_hand, num of colors, and num of value, so 4*3*3 = 36. Same with opp_clue

Input layer = 15+9+15+36+36 = 111 node
also add in num bombs, number of cards left, number of clues, 4, 15-4*2=7, 8

total input layer = 111+4+7+8 = 130


In [1]:
import random
import numpy as np
from memory import Memory
from Game import *
import pickle
from Model import *
import config
import pandas as pd
from collections import Counter
from Player import *


C:\Users\yidin\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [36]:
def create_GS(deck, p1_hand, p2_hand, discard, played):
    p1 = Player("P1", deck.copy(), nn_model())
    p2 = Player("P2", deck.copy(), nn_model())

    board = Board(deck.copy())

    played_cards = []

    for slot in played:
        i = 1
        num = played[slot]
        while i <= num:
            played_cards.append(str(i)+slot)
            i+=1


    for card in p1_hand:
        p1.draw([deck.remove(card)])

    for card in p2_hand:
        p2.draw([deck.remove(card)])

    for card in played_cards:
        play = deck.remove(card)
        board.play_card(play)
    
    for card in discard:
        play = deck.remove(card)
        board.discard(play)
        
    return(p1, p2, board)

In [37]:
scores = []
colors = ["B","G","R","W","Y"]
numbers = [1,1,1,2,2,3,3,4,4,5]

cards = [Card(value, color) for color in colors for value in numbers]
deck = Deck(cards = cards)

p1_hand = ["1B","3W","4Y", "3Y"]
p2_hand = ["1B","2B","4Y", "3Y"]

discard = ["5Y","2Y"]
played ={
    "B":2,
    "G":1,
    "R":0,
    "W":0,
    "Y":1
        }

p1,p2,board = create_GS(deck, p1_hand, p2_hand, discard, played)


In [42]:
nn_inputs = p1.nn_input(p2,board)

In [44]:
pub = nn_inputs["public"]
pub

[array([[[0.04081633, 0.06122449, 0.06122449, 0.06122449, 0.06122449],
         [0.04081633, 0.04081633, 0.04081633, 0.04081633, 0.04081633],
         [0.04081633, 0.04081633, 0.04081633, 0.04081633, 0.04081633],
         [0.04081633, 0.04081633, 0.04081633, 0.04081633, 0.04081633],
         [0.02040816, 0.02040816, 0.02040816, 0.02040816, 0.02040816]],
 
        [[1.        , 1.        , 0.        , 0.        , 1.        ],
         [1.        , 0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ]],
 
        [[0.        , 0.        , 1.        , 1.        , 0.        ],
         [0.        , 1.        , 0.        , 0.        , 1.        ],
         [1.        , 0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ],


In [84]:
import numpy as np
np.random.seed(1337)
from keras.layers import Input, Dense, Conv1D, Conv2D, Flatten, concatenate
from keras.models import Model

# This returns a tensor
input1 = Input(shape=(3,5,5))
input2 = Input(shape=(3,5,5))
input3 = Input(shape=(3,5,5))
input4 = Input(shape=(3,5,5))

# a layer instance is callable on a tensor, and returns a tensor
conv = Conv2D(1, kernel_size = (3,5), activation='linear')
card1 = conv(input1)
card2 = conv(input2)
card3 = conv(input3)
card4 = conv(input4)

merged = concatenate([card1, card2, card3, card4], axis=-1)
value = Dense(1, activation = 'linear')(merged)
# This creates a model that includes
# the Input layer and three Dense layers

model = Model(inputs=[input1, input2, input3, input4], outputs= value)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

data = pub
data

[array([[[0.04081633, 0.06122449, 0.06122449, 0.06122449, 0.06122449],
         [0.04081633, 0.04081633, 0.04081633, 0.04081633, 0.04081633],
         [0.04081633, 0.04081633, 0.04081633, 0.04081633, 0.04081633],
         [0.04081633, 0.04081633, 0.04081633, 0.04081633, 0.04081633],
         [0.02040816, 0.02040816, 0.02040816, 0.02040816, 0.02040816]],
 
        [[1.        , 1.        , 0.        , 0.        , 1.        ],
         [1.        , 0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ]],
 
        [[0.        , 0.        , 1.        , 1.        , 0.        ],
         [0.        , 1.        , 0.        , 0.        , 1.        ],
         [1.        , 0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ],


In [80]:
from keras import backend as K

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    # evaluation functions

# Testing
test = [data[0][np.newaxis],data[1][np.newaxis]]
layer_outs = [func([test, 1.]) for func in functors]

for l in layer_outs:
    print(l)
    
weights = model.layers[1].get_weights()[0]
print("\n Weights: \n",weights)

AttributeError: Layer conv2d_18 has multiple inbound nodes, hence the notion of "layer output" is ill-defined. Use `get_output_at(node_index)` instead.

In [85]:
model.predict([data[0][np.newaxis],data[1][np.newaxis],data[2][np.newaxis],data[3][np.newaxis]])

array([[[[0.600511]]]], dtype=float32)